In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [2]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = '/content/drive/MyDrive/Colab Notebooks/FinalProject/model/best_model_chargeBalance.dat'

# 교차검증 횟수
cv_count = 2

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들기
f1_score_list = []
accuracy_score_list = []

# 학습 모델 이름
model_name_list = []

In [5]:
# train 파일 병합
# CSV 파일 불러오기
claim_train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/data/피처선택완료_데이터/chargeInfo_train.csv')  # 청구정보
balance_train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/data/피처선택완료_데이터/balanceInfo_train.csv')  # 잔액정보

# 중복컬럼 제거
claim_train_df = claim_train_df.drop(columns=['Segment'])

# 기준년월, ID 기준으로 병합
train_df = pd.merge(claim_train_df, balance_train_df, on=['ID', '기준년월'], how='left')

# 병합 결과 확인
print(train_df.shape)
display(train_df.head())

(2400000, 28)


,기준년월,ID,청구금액_R3M,포인트_포인트_건별_R3M,포인트_마일리지_월적립_R3M,포인트_포인트_월적립_R3M,포인트_적립포인트_R3M,포인트_이용포인트_R3M,마일_적립포인트_R3M,마일_이용포인트_R3M,...,월중평잔_일시불,평잔_일시불_6M,평잔_RV일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_해외_6M,잔액_할부_B0M,평잔_할부_6M,잔액_현금서비스_B0M,평잔_CA_6M,평잔_6M
0,201807,TRAIN_000000,46588,1827,0,0,1827,1296,0,0,...,1503,2440,0,0,0,962,572,22971,17008,15988
1,201807,TRAIN_000001,10530,0,0,0,0,0,0,0,...,4447,2677,2830,0,0,2390,2736,0,0,7045
2,201807,TRAIN_000002,85931,5153,0,0,5153,2765,0,0,...,5540,9118,8870,0,0,5113,4429,21531,43351,66549
3,201807,TRAIN_000003,61518,2523,0,0,2523,2467,0,0,...,606,884,0,0,0,5025,5097,26284,30697,30139
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,21,0,0,0,0,0,0,0,28


In [6]:
# test 파일 병합
# CSV 파일 불러오기
claim_test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/data/피처선택완료_데이터/chargeInfo_test.csv')  # 청구정보
balance_test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/data/피처선택완료_데이터/balanceInfo_test.csv')  # 잔액정보

# 기준년월, ID 기준으로 병합
test_df = pd.merge(claim_test_df, balance_test_df, on=['ID', '기준년월'], how='left')

# 3. 병합 결과 확인
print(test_df.shape)
display(test_df.head())


(600000, 27)


,기준년월,ID,청구금액_R3M,포인트_포인트_건별_R3M,포인트_마일리지_월적립_R3M,포인트_포인트_월적립_R3M,포인트_적립포인트_R3M,포인트_이용포인트_R3M,마일_적립포인트_R3M,마일_이용포인트_R3M,...,월중평잔_일시불,평잔_일시불_6M,평잔_RV일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_해외_6M,잔액_할부_B0M,평잔_할부_6M,잔액_현금서비스_B0M,평잔_CA_6M,평잔_6M
0,201807,TEST_00000,11441,0,0,0,0,0,2532,2604,...,5187,2933,0,384,0,0,0,0,0,3847
1,201807,TEST_00001,20522,0,0,0,0,0,0,0,...,865,657,0,0,0,8628,7979,5448,1530,42634
2,201807,TEST_00002,50508,0,0,6032,6032,5945,0,0,...,5591,6253,0,0,0,6144,12329,0,0,22936
3,201807,TEST_00003,4604,1319,0,0,1319,1059,0,0,...,1545,819,0,0,0,945,468,0,0,1206
4,201807,TEST_00004,6788,0,0,0,0,0,0,0,...,1462,979,0,0,0,2558,2414,0,0,2914


In [7]:
# 데이터 프레임을 합친다.
all_df = pd.concat([train_df, test_df])
all_df.reset_index(inplace=True, drop=True)
all_df

,기준년월,ID,청구금액_R3M,포인트_포인트_건별_R3M,포인트_마일리지_월적립_R3M,포인트_포인트_월적립_R3M,포인트_적립포인트_R3M,포인트_이용포인트_R3M,마일_적립포인트_R3M,마일_이용포인트_R3M,...,월중평잔_일시불,평잔_일시불_6M,평잔_RV일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_해외_6M,잔액_할부_B0M,평잔_할부_6M,잔액_현금서비스_B0M,평잔_CA_6M,평잔_6M
0,201807,TRAIN_000000,46588,1827,0,0,1827,1296,0,0,...,1503,2440,0,0,0,962,572,22971,17008,15988
1,201807,TRAIN_000001,10530,0,0,0,0,0,0,0,...,4447,2677,2830,0,0,2390,2736,0,0,7045
2,201807,TRAIN_000002,85931,5153,0,0,5153,2765,0,0,...,5540,9118,8870,0,0,5113,4429,21531,43351,66549
3,201807,TRAIN_000003,61518,2523,0,0,2523,2467,0,0,...,606,884,0,0,0,5025,5097,26284,30697,30139
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,21,0,0,0,0,0,0,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2999996,201812,TEST_99996,1256,0,0,0,0,0,0,0,...,256,137,0,0,0,0,0,0,0,275
2999997,201812,TEST_99997,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2999998,201812,TEST_99998,48141,0,0,0,0,3804,0,0,...,14005,18108,0,240,0,2786,1830,0,0,20549


In [8]:
# 결과 데이터는 제거한다.
all_df.drop('Segment', axis=1, inplace=True)
all_df

,기준년월,ID,청구금액_R3M,포인트_포인트_건별_R3M,포인트_마일리지_월적립_R3M,포인트_포인트_월적립_R3M,포인트_적립포인트_R3M,포인트_이용포인트_R3M,마일_적립포인트_R3M,마일_이용포인트_R3M,...,월중평잔_일시불,평잔_일시불_6M,평잔_RV일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_해외_6M,잔액_할부_B0M,평잔_할부_6M,잔액_현금서비스_B0M,평잔_CA_6M,평잔_6M
0,201807,TRAIN_000000,46588,1827,0,0,1827,1296,0,0,...,1503,2440,0,0,0,962,572,22971,17008,15988
1,201807,TRAIN_000001,10530,0,0,0,0,0,0,0,...,4447,2677,2830,0,0,2390,2736,0,0,7045
2,201807,TRAIN_000002,85931,5153,0,0,5153,2765,0,0,...,5540,9118,8870,0,0,5113,4429,21531,43351,66549
3,201807,TRAIN_000003,61518,2523,0,0,2523,2467,0,0,...,606,884,0,0,0,5025,5097,26284,30697,30139
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,21,0,0,0,0,0,0,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2999996,201812,TEST_99996,1256,0,0,0,0,0,0,0,...,256,137,0,0,0,0,0,0,0,275
2999997,201812,TEST_99997,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2999998,201812,TEST_99998,48141,0,0,0,0,3804,0,0,...,14005,18108,0,240,0,2786,1830,0,0,20549


In [9]:
df1 = train_df.drop(columns=['기준년월','ID'])
df1

,청구금액_R3M,포인트_포인트_건별_R3M,포인트_마일리지_월적립_R3M,포인트_포인트_월적립_R3M,포인트_적립포인트_R3M,포인트_이용포인트_R3M,마일_적립포인트_R3M,마일_이용포인트_R3M,할인건수_R3M,할인금액_R3M,...,월중평잔_일시불,평잔_일시불_6M,평잔_RV일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_해외_6M,잔액_할부_B0M,평잔_할부_6M,잔액_현금서비스_B0M,평잔_CA_6M,평잔_6M
0,46588,1827,0,0,1827,1296,0,0,0,0,...,1503,2440,0,0,0,962,572,22971,17008,15988
1,10530,0,0,0,0,0,0,0,0,0,...,4447,2677,2830,0,0,2390,2736,0,0,7045
2,85931,5153,0,0,5153,2765,0,0,0,0,...,5540,9118,8870,0,0,5113,4429,21531,43351,66549
3,61518,2523,0,0,2523,2467,0,0,0,0,...,606,884,0,0,0,5025,5097,26284,30697,30139
4,0,0,0,0,0,0,0,0,0,0,...,0,21,0,0,0,0,0,0,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,0,0,0,0,0,2049,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,37515,10890,0,0,8478,13202,0,0,0,0,...,5515,12524,0,0,0,0,0,0,0,67826
2399997,22274,0,0,0,0,0,0,0,0,0,...,3046,3241,0,0,0,2960,3995,0,0,8627
2399998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# 입력 변수(X)와 타깃 변수(y)로 데이터를 분리한다.
X = df1.drop('Segment', axis=1)
y = df1['Segment']

# 문자열 -> 숫자
encoder1 = LabelEncoder()
encoder1.fit(y)
y2 = encoder1.transform(y)

# 입력 데이터 표준화
scaler1 = StandardScaler()
scaler1.fit(X)
X2 = scaler1.transform(X)

# 학습용과 검증용으로 나눈다.
train_X = X2
train_y = y2

### LogisticRegression

In [14]:
lr_basic_model = LogisticRegression(multi_class='multinomial')

# 모델 학습 수행
lr_basic_model.fit(train_X, train_y)

LogisticRegression(multi_class='multinomial')

In [15]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [16]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

{np.int64(0): np.int64(55), np.int64(1): np.int64(1), np.int64(2): np.int64(14523), np.int64(3): np.int64(46817), np.int64(4): np.int64(538604)}


In [ ]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

{'A': np.int64(55), 'B': np.int64(1), 'C': np.int64(14523), 'D': np.int64(46817), 'E': np.int64(538604)}


In [ ]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [ ]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/sample_submission.csv')

In [ ]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('charBal_submit.csv', index=False)

In [ ]:
# 최종 제출 파일 내용 출력
submit

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C


### DecisionTree

In [ ]:
tree_basic_model = DecisionTreeClassifier()

# 모델 학습 수행
tree_basic_model.fit(train_X, train_y)

DecisionTreeClassifier()

In [ ]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = train_df.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

KeyError: "['기준년월', 'ID'] not in index"

In [ ]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

In [ ]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

In [ ]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [ ]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('open/sample_submission.csv')

In [ ]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('tree_submit.csv', index=False)

In [ ]:
# 최종 제출 파일 내용 출력
submit

### AdaBoost

In [ ]:
ada_basic_model = AdaBoostClassifier()

# 모델 학습 수행
ada_basic_model.fit(train_X, train_y)

In [ ]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [ ]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

In [ ]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

In [ ]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [ ]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('open/sample_submission.csv')

In [ ]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('ada_submit.csv', index=False)

In [ ]:
# 최종 제출 파일 내용 출력
submit

### LGBM

In [ ]:
lgbm_basic_model = LGBMClassifier(verbose=-1)

# 모델 학습 수행
lgbm_basic_model.fit(train_X, train_y)

In [ ]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [ ]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

In [ ]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

In [ ]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [ ]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('open/sample_submission.csv')

In [ ]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('lgbm_submit.csv', index=False)

In [ ]:
# 최종 제출 파일 내용 출력
submit

### XGBoost

In [11]:
xgboost_basic_model = XGBClassifier(verbose=-1, silent=True, tree_method='gpu_hist')

# 모델 학습 수행
xgboost_basic_model.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [17]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

{np.int64(0): np.int64(55), np.int64(1): np.int64(1), np.int64(2): np.int64(14523), np.int64(3): np.int64(46817), np.int64(4): np.int64(538604)}


In [18]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

{'A': np.int64(55), 'B': np.int64(1), 'C': np.int64(14523), 'D': np.int64(46817), 'E': np.int64(538604)}


In [19]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [20]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/sample_submission.csv')

In [23]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/xg_submit.csv', index=False)

In [22]:
# 최종 제출 파일 내용 출력
submit

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C


### RandomForest

In [ ]:
rf_basic_model = RandomForestClassifier()

# 모델 학습 수행
rf_basic_model.fit(train_X, train_y)

In [ ]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, 기준년월, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', '기준년월', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [ ]:
# 모델 예측 수행
pred_y_numeric = lr_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

In [ ]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

In [ ]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [ ]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('open/sample_submission.csv')

In [ ]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('rf_submit.csv', index=False)

In [ ]:
# 최종 제출 파일 내용 출력
submit